In [1]:
import numpy as np
import pandas as pd
from scipy.optimize import minimize
from scipy.special import expit  # sigmoid

import importlib
import utils.data_processing as data_processing
importlib.reload(data_processing)
from utils.data_processing import set_seed, save_output, get_df

# overall / first

In [16]:
df = get_df()
# df = df.groupby('subject').head(1).reset_index(drop=True) # first trial only

In [17]:
# --- Features ---
df["dR"] = df["m2"] - df["m1"]

def compute_dI(uc):
    if uc == 1: return -1   # A once, B thrice → B less informative
    if uc == 3: return 1    # A thrice, B once → B more informative
    return 0                # both twice
df["dI"] = df["uc"].apply(compute_dI)

df["dS"] = 0  # optional (no side info)
df["choice"] = df["c4"]      # first free choice
df["horizon"] = df["gameLength"]

In [18]:
def neg_log_likelihood(params, dR, dI, dS, choice):
    alpha, bias, log_sigma = params
    sigma = np.exp(log_sigma)
    qdiff = dR + alpha * dI + bias * dS
    p_chooseB = expit(qdiff / sigma)
    eps = 1e-9
    loglik = choice * np.log(p_chooseB + eps) + (1 - choice) * np.log(1 - p_chooseB + eps)
    return -np.sum(loglik)

results = {}
for horizon in [1, 6]:
    subset = df[df["horizon"] == horizon]
    x0 = [0, 0, 0]  # alpha, bias, log_sigma initial guess
    res = minimize(neg_log_likelihood, x0,
                   args=(subset["dR"], subset["dI"], subset["dS"], subset["choice"]))
    results[horizon] = res.x
    print(f"H{horizon} α={res.x[0]:.3f}, bias={res.x[1]:.3f}, σ={np.exp(res.x[2]):.3f}")



H1 α=-2.686, bias=0.000, σ=10.899
H6 α=2.760, bias=0.000, σ=15.667


In [19]:
def predict_acc(params, df):
    alpha, bias, log_sigma = params
    sigma = np.exp(log_sigma)
    qdiff = df["dR"] + alpha * df["dI"] + bias * df["dS"]
    preds = expit(qdiff / sigma) > 0.5
    return np.mean(preds == df["choice"])

for horizon in [1, 6]:
    subset = df[df["horizon"] == horizon]
    acc = predict_acc(results[horizon], subset)
    print(f"H{horizon} accuracy: {acc:.3f}")



H1 accuracy: 0.757
H6 accuracy: 0.699


In [20]:
# --- Overall accuracy (use correct params for each row) ---
all_preds, all_true = [], []
for horizon in [1, 6]:
    subset = df[df["horizon"] == horizon]
    params = results[horizon]
    alpha, bias, log_sigma = params
    sigma = np.exp(log_sigma)
    qdiff = subset["dR"] + alpha * subset["dI"] + bias * subset["dS"]
    preds = expit(qdiff / sigma) > 0.5
    all_preds.extend(preds)
    all_true.extend(subset["choice"].values)

overall_acc = np.mean(np.array(all_preds) == np.array(all_true))
print(f"Overall accuracy: {overall_acc:.3f}")

Overall accuracy: 0.728


In [21]:
def compute_nll(df, params):
    alpha, bias, log_sigma = params
    sigma = np.exp(log_sigma)

    qdiff = df["dR"] + alpha * df["dI"] + bias * df["dS"]
    prob = expit(qdiff / sigma)

    eps = 1e-9
    # if choice = 1 → use prob; if 0 → use (1-prob)
    loglik = df["choice"] * np.log(prob + eps) + (1 - df["choice"]) * np.log(1 - prob + eps)
    return -np.mean(loglik)   # mean NLL per trial


In [23]:
nll_h1 = compute_nll(df[df.horizon == 1], results[1])
nll_h6 = compute_nll(df[df.horizon == 6], results[6])

# overall NLL by merging both sets
df_all = df.copy()
df_all["params"] = df_all["horizon"].apply(lambda h: results[h])
nll_overall = np.mean([
    compute_nll(df[df.horizon == h], results[h]) for h in [1, 6]
])

print(f"H1 NLL: {nll_h1:.3f}, H6 NLL: {nll_h6:.3f}, Overall NLL: {nll_overall:.3f}")

H1 NLL: 0.526, H6 NLL: 0.595, Overall NLL: 0.561


# subject

In [8]:
def subject_accuracy(df, params):
    alpha, bias, log_sigma = params
    sigma = np.exp(log_sigma)
    qdiff = df["dR"] + alpha * df["dI"] + bias * df["dS"]
    preds = expit(qdiff / sigma) > 0.5
    return np.mean(preds == df["choice"])

subject_accs = {1: [], 6: []}

for horizon in [1, 6]:
    sub_df = df[df["horizon"] == horizon]
    for subj, group in sub_df.groupby("subject"):
        acc = subject_accuracy(group, results[horizon])
        subject_accs[horizon].append(acc)

# mean and sem (or sd)
for horizon in [1, 6]:
    arr = np.array(subject_accs[horizon])
    mean_acc = arr.mean()
    sem_acc = arr.std(ddof=1) / np.sqrt(len(arr))
    print(f"H{horizon}: mean={mean_acc:.3f}, sem={sem_acc:.3f}, range=({arr.min():.3f}–{arr.max():.3f})")

H1: mean=0.761, sem=0.003, range=(0.150–1.000)
H6: mean=0.704, sem=0.003, range=(0.156–1.000)


In [9]:
overall_subject_accs = []

for subj, group in df.groupby("subject"):
    preds_all = []
    true_all = []

    # H1 games
    g1 = group[group["horizon"] == 1]
    if len(g1) > 0:
        alpha, bias, log_sigma = results[1]
        sigma = np.exp(log_sigma)
        qdiff = g1["dR"] + alpha * g1["dI"] + bias * g1["dS"]
        preds_all.extend((expit(qdiff / sigma) > 0.5).astype(int))
        true_all.extend(g1["choice"].values)

    # H6 games
    g6 = group[group["horizon"] == 6]
    if len(g6) > 0:
        alpha, bias, log_sigma = results[6]
        sigma = np.exp(log_sigma)
        qdiff = g6["dR"] + alpha * g6["dI"] + bias * g6["dS"]
        preds_all.extend((expit(qdiff / sigma) > 0.5).astype(int))
        true_all.extend(g6["choice"].values)

    # final per-subject overall accuracy
    preds_all = np.array(preds_all)
    true_all = np.array(true_all)
    overall_subject_accs.append(np.mean(preds_all == true_all))


In [10]:
overall_arr = np.array(overall_subject_accs)
overall_mean = overall_arr.mean()
overall_sem = overall_arr.std(ddof=1) / np.sqrt(len(overall_arr))

print(f"Overall subject mean={overall_mean:.3f}, sem={overall_sem:.3f}, "
      f"range=({overall_arr.min():.3f}–{overall_arr.max():.3f})")


Overall subject mean=0.733, sem=0.003, range=(0.153–0.962)
